1. 記述子の選択
2. アルゴリズムの選択
3. dbから予測データを取得
4. ADの適用範囲を調査

In [1]:
import pandas as pd

In [19]:
df = pd.read_csv('database_1114.csv')
print(df.columns)

Index(['entry', 'R1-', 'R2', 'organocatalyst', 'organocatalyst(mol%)', 'under',
       'temp(℃)', 'time(h)', 'Cu(OAc)2(mol%)', 'AcOH(mol%)', '収率(%)'],
      dtype='object')


In [20]:
# 基質、触媒、収率で欠損がある場合は削除
df.dropna(subset=['R1-', 'R2', 'organocatalyst', '収率(%)'], inplace=True)

In [21]:
print('欠損値合計', df.isnull().sum())
na_row = df.isnull().any(axis=1)
df.loc[na_row, :]

欠損値合計 entry                   0
R1-                     0
R2                      0
organocatalyst          0
organocatalyst(mol%)    0
under                   0
temp(℃)                 0
time(h)                 0
Cu(OAc)2(mol%)          0
AcOH(mol%)              0
収率(%)                   0
dtype: int64


,entry,R1-,R2,organocatalyst,organocatalyst(mol%),under,temp(℃),time(h),Cu(OAc)2(mol%),AcOH(mol%),収率(%)


In [28]:
# R2, underはカテゴリ値で置く
# 念の為確認。
[print(df[name].value_counts()) for name in df.columns if name in ['R2', 'under']]

H     101
C      61
Ph      3
Name: R2, dtype: int64
O2     111
air     53
Ar       1
Name: under, dtype: int64


[None, None]

In [32]:
#カテゴリ値に変換
df = pd.get_dummies(df, columns=['R2','under'])
df.head(5)

KeyError: "None of [Index(['R2', 'under'], dtype='object')] are in the [columns]"

In [79]:
# tempの「rt」を23℃に変換
df['temp(℃)'][df['temp(℃)'] == 'rt'] = 23
print(df['temp(℃)'] == 'rt')

0      False
1      False
2      False
3      False
4      False
       ...  
206    False
207    False
208    False
209    False
212    False
Name: temp(℃), Length: 165, dtype: bool


/var/folders/12/6r_lrg9d6kx949nw6gcq9ksh0000gn/T/ipykernel_93936/2666549685.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['temp(℃)'][df['temp(℃)'] == 'rt'] = 23


In [215]:
check_columns = df.columns.drop(['entry', '収率(%)'])

# 説明変数のみでみた時の重複削除
_df = df.drop(columns='entry').drop_duplicates(subset=check_columns).reset_index(drop=True)
X = _df.drop(columns=['収率(%)'])
y = _df['収率(%)']
_df.to_csv('exec_1114_前処理後.csv')

In [238]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
display(X_train.shape)
display(X_test.shape)

(123, 13)

(31, 13)

In [262]:
# 説明変数をスケーリング
# 目的変数は必要ない。
# https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re
from sklearn.preprocessing import StandardScaler
scaling_columns = ['organocatalyst(mol%)','temp(℃)','time(h)','Cu(OAc)2(mol%)', 'AcOH(mol%)']
scaler = StandardScaler()
scaler.fit(X_train[scaling_columns])
X_train_scaled = pd.concat([X_train.drop(columns=scaling_columns), 
                           pd.DataFrame(scaler.transform(X_train[scaling_columns]),index=X_train.index , columns=scaling_columns)],
                           axis=1, join='inner')
X_test_scaled = pd.concat([X_test.drop(columns=scaling_columns),
                           pd.DataFrame(scaler.transform(X_test[scaling_columns]),index=X_test.index,columns=scaling_columns)],
                           axis=1, join='inner')

print(X_train_scaled.shape)
print(X_test_scaled.shape)

(123, 13)
(31, 13)


# 記述子の選定
1. 全てのパラメータを用いる。
   基質R1と触媒の化学構造をmorgan記述子に変換する。
2. 基質R1,R2と触媒のデータのみ用いる。(重複は削除)
3. 類似度(基質と触媒のMorgan記述子に基づいて計算したタニモト係数)とR2
4. DFTから出す？？？

# モデルの選定
- PLS

  多重共線性。寄与率見れる。
  相関関係がある複数の予測子変数が含まれているデータに用いる

- LASSO

  残差二乗和に罰則項を設け、過剰適合を防ぐ
  
- SVR

  説明変数を非線形変換した後の「特徴量空間」の中で誤差を少なくする。 => カーネル関数を用いて類似度を測定している。
  精度が高い
  
- GPR
  
- RF

  ブースティングを使ったアンサンブル決定木、精度が高い、結果がわかりやすい。
  
- GBDT

  勾配ブースティング
  
- XGB

外側のクロスバリデーションは一つ抜き。内側のバリデーションは5分割のクロスバリデーションを用いたダブルクロスバリデーションでモデルをトレーニングした。

In [301]:
X_train_scaled.columns

Index(['R1-', 'organocatalyst', 'R2_C', 'R2_H', 'R2_Ph', 'under_Ar',
       'under_O2', 'under_air', 'organocatalyst(mol%)', 'temp(℃)', 'time(h)',
       'Cu(OAc)2(mol%)', 'AcOH(mol%)'],
      dtype='object')

In [399]:
BIT = 2048

In [453]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs

def toFinger(df, columns):
    df_copy = df.drop(columns=columns)
    for column in columns:
        fingerprints = toFingerFromSmiles(df[column])
        column_names = list(map(lambda x: str(x)+'_'+column, range(BIT)))
        df_copy = pd.merge(df_copy, pd.DataFrame(fingerprints,index=df.index, columns=column_names), left_index=True, right_index=True)
    return df_copy

def toFingerFromSmiles(series):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    fingerprints = []
    for mol_idx, mol in enumerate(mols):
        try:
            # listに直してる。
            fingerprint = [x for x in Chem.AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT)]
            fingerprints.append(fingerprint)
        except Exception as e:
            print("Error", mol_idx)
            break
    return fingerprints
del list

In [353]:
# 1. 全てのデータを使用
finger_columns = ['R1-', 'organocatalyst']
X_train_1 = toFinger(X_train_scaled, finger_columns)
X_test_1 = toFinger(X_test_scaled, finger_columns)

In [354]:
display(X_train_1.head(3))
display(X_test_1.head(3))

,R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air,organocatalyst(mol%),temp(℃),time(h),Cu(OAc)2(mol%),...,2038_organocatalyst,2039_organocatalyst,2040_organocatalyst,2041_organocatalyst,2042_organocatalyst,2043_organocatalyst,2044_organocatalyst,2045_organocatalyst,2046_organocatalyst,2047_organocatalyst
19,0,1,0,0,1,0,-0.489763,1.266443,-1.214437,-0.260514,...,0,0,0,0,0,0,0,0,0,0
138,1,0,0,0,0,1,0.816980,-0.843271,-0.270349,-0.260514,...,0,0,0,0,0,0,0,0,0,0
95,0,1,0,0,1,0,-1.273809,-0.843271,0.988436,2.078405,...,0,0,0,0,0,0,0,0,0,0


,R2_C,R2_H,R2_Ph,under_Ar,under_O2,under_air,organocatalyst(mol%),temp(℃),time(h),Cu(OAc)2(mol%),...,2038_organocatalyst,2039_organocatalyst,2040_organocatalyst,2041_organocatalyst,2042_organocatalyst,2043_organocatalyst,2044_organocatalyst,2045_organocatalyst,2046_organocatalyst,2047_organocatalyst
26,0,1,0,0,1,0,-1.796506,-0.843271,-1.214437,-0.260514,...,0,0,0,0,0,0,0,0,0,0
89,0,1,0,0,1,0,-0.489763,-0.843271,0.988436,2.078405,...,0,0,0,0,0,0,0,0,0,0
151,0,0,1,0,1,0,0.816980,0.321795,2.247221,-0.260514,...,0,0,0,0,0,0,0,0,0,0


In [397]:
# 基質R1,R2と触媒のデータのみ用いる。(重複は削除)
drop_columns = ['under_Ar', 'under_O2', 'under_air', 'organocatalyst(mol%)', 'temp(℃)', 'time(h)','Cu(OAc)2(mol%)', 'AcOH(mol%)']

X_train_1_dropped = X_train_1.drop(columns=drop_columns)
X_test_1_dropped = X_test_1.drop(columns=drop_columns)

train_set_2 = pd.concat([X_train_1_dropped, y_train], axis=1)
train_set_2.drop_duplicates(subset=X_train_1_dropped.columns, inplace=True)
print('trainのduplicated: ', train_set_2.duplicated().value_counts())
X_train_2 = train_set_2.drop(columns=['収率(%)'])
y_train_2 = train_set_2['収率(%)']

test_set_2 = pd.concat([X_test_1_dropped, y_test], axis=1)
test_set_2.drop_duplicates(subset=X_test_1_dropped.columns, inplace=True)
print('trainのduplicated: ', test_set_2.duplicated().value_counts())
X_test_2 = test_set_2.drop(columns=['収率(%)'])
y_test_2 = test_set_2['収率(%)']

trainのduplicated:  False    94
dtype: int64
trainのduplicated:  False    26
dtype: int64


In [398]:
display('X_train_2: ', X_train_2.head(3))
display('X_test_2: ', X_test_2.head(3))
display('y_train_2: ', y_train_2.head(3))
display('y_test_2: ', y_test_2.head(3))

'X_train_2: '

,R2_C,R2_H,R2_Ph,0_R1-,1_R1-,2_R1-,3_R1-,4_R1-,5_R1-,6_R1-,...,2038_organocatalyst,2039_organocatalyst,2040_organocatalyst,2041_organocatalyst,2042_organocatalyst,2043_organocatalyst,2044_organocatalyst,2045_organocatalyst,2046_organocatalyst,2047_organocatalyst
19,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
95,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


'X_test_2: '

,R2_C,R2_H,R2_Ph,0_R1-,1_R1-,2_R1-,3_R1-,4_R1-,5_R1-,6_R1-,...,2038_organocatalyst,2039_organocatalyst,2040_organocatalyst,2041_organocatalyst,2042_organocatalyst,2043_organocatalyst,2044_organocatalyst,2045_organocatalyst,2046_organocatalyst,2047_organocatalyst
26,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
151,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


'y_train_2: '

19     88.0
138    19.0
95     56.0
Name: 収率(%), dtype: float64

'y_test_2: '

26      6.0
89     90.0
151    92.0
Name: 収率(%), dtype: float64

In [494]:
# 類似性(基質と触媒のMorgan記述子に基づいて計算したタニモト係数)をxに追加
# def toTanimoto(df):
#     df_converted = pd.DataFrame()
#     for i in range(0, BIT * 2, BIT):
#         fingers = df.iloc[:, i:i+BIT].values.tolist()
#         tanimoto_rows = [
#         df_converted = pd.concat(df_converted, pd.DataFrame(tanimoto_rows, columns=fingers.columns), axis=1)
#     return df_converted

def toTanimoto(series, suffix):
    mols = []
    for smile in series:
        if smile in ['-', 0]: smile = '' 
        mols.append(Chem.MolFromSmiles(smile))

    fps = [Chem.AllChem.GetMorganFingerprintAsBitVect(mol,2,BIT) for mol in mols]
    column_names = [str(i)+'-'+suffix for i in range(len(fps))]
    return pd.DataFrame([DataStructs.BulkTanimotoSimilarity(fp, fps) for fp in fps], columns=column_names)

method_3_selected = ['R1-', 'organocatalyst', 'R2_C', 'R2_H', 'R2_Ph']
train_set_3 = pd.concat([X_train, y_train], axis=1)[method_3_selected + ['収率(%)']]
train_set_3.drop_duplicates(subset=method_3_selected,inplace=True)
test_set_3 = pd.concat([X_test, y_test], axis=1)[method_3_selected + ['収率(%)']]
train_set_3.drop_duplicates(subset=method_3_selected,inplace=True)

X_train_3 = pd.concat([toTanimoto(train_set_3['R1-'],'R1'), toTanimoto(train_set_3['organocatalyst'], 'organocatalyst'), train_set_3[['R2_C','R2_H','R2_Ph']]], axis=1)
y_train = train_set_3['収率(%)']
X_test_3 = pd.concat([toTanimoto(test_set_3['R1-'],'R1'), toTanimoto(test_set_3['organocatalyst'], 'organocatalyst'), train_set_3[['R2_C','R2_H','R2_Ph']]], axis=1)
y_test = test_set_3['収率(%)']

In [ ]:
# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
rf = RandomForestRegressor()
rf.fit(X_train_1,Y_train_1)
r2_score(rf.predict(X_test_1))


In [ ]:
PLS
LASSO
SVR
GPR
GBDT
XGB